In [3]:
%%capture
! pip install pyLDAvis
! pip install pymorphy2

In [33]:
from gensim.test.utils import common_corpus
from gensim.models import LdaSeqModel
import gensim
import pickle
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
import warnings
import json
import pymorphy2
import nltk
import re 
from tqdm.notebook import tqdm
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
import nltk
from nltk import ngrams
import time
import dateutil.parser as dp
nltk.download('stopwords')
import pandas as pd
from sklearn.model_selection import train_test_split
import datetime
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from typing import List
import numpy as np

russian_stopwords = stopwords.words("russian")
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

[nltk_data] Error loading stopwords: <urlopen error [Errno 104]
[nltk_data]     Connection reset by peer>


In [75]:
row_data = pd.read_json('/home/jovyan/work/outputs_upd.jsonl', lines = True, orient = 'records')

In [76]:
class Preparator():
    def __init__(self):
        self.morph = pymorphy2.MorphAnalyzer()
        self.tokenizer = nltk.WordPunctTokenizer()
        self.regexp = re.compile('[-%-,_\.\n\\\t\r/\W]')  # оставляет только буквы и цифры
        self.pos_decoder = {"ADJF": "ADJ",
                            'ADJS': 'ADJ',
                            'COMP': 'ADJ',
                            'INFN': 'VERB',
                            'PRTF': 'VERB'}

    def norm_word(self, word: str) -> str:
        w = str(re.sub(' +', ' ', self.regexp.sub(' ', word.lower()))).strip()
        if len(w) > 1 and w[0].isalpha():
            parsed = self.morph.parse(w)[0]
            tag = str(parsed.tag.POS)
            if tag not in ['NPRO', 'PRED', 'PREP', 'CONJ', 'PRCL', 'INTJ', 'VERB']:
                tag = tag if tag not in self.pos_decoder else self.pos_decoder[tag]
                return parsed.normal_form.replace('ё', 'e')
        return ''
    
    def norm_sentence(self, sentence: str):
        tokens = list(filter(lambda x: len(x), [self.norm_word(word) for word in self.tokenizer.tokenize(sentence.lower())]))
        # bigrams = ['_'.join(w) for w in  ngrams(tokens,n=2)]
        return tokens
    
        

In [77]:
def extract_keywords(tokens: List[str],
                     embeddings: KeyedVectors,
                     top_n: int = 5,
                     diversity: float = 0.5,
                     nr_candidates: int = 20) -> str:
    candidates = []
    doc_embedding = text2vec(tokens, embeddings).reshape(1, -1)
    candidate_embeddings = []
    for candidate in tokens:
        if candidate in embeddings:
            candidates.append(candidate)
            candidate_embeddings.append(embeddings[candidate])

    if len(candidates) == 0:
        return tokens

    # Calculate distances and extract keywords
    distances = cosine_similarity(np.nan_to_num(doc_embedding), np.nan_to_num(candidate_embeddings))
    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]][::-1]

    return list(set(keywords))

def text2vec(tokens: List[str], embeddings: KeyedVectors) -> np.ndarray:
    relevant = 0
    words_vecs = np.zeros((embeddings.vector_size,))
    for word in tokens:
        if word in embeddings:
            words_vecs += embeddings.word_vec(word, True)
            relevant += 1

    if relevant:
        words_vecs /= relevant
    return words_vecs

In [78]:
W2V_SOURCE = "/home/jovyan/work/custom_word2vec_uncod.bin" # Путь к файлу custom_word2vec_uncod.bin
prep = Preparator()
w2v = KeyedVectors.load_word2vec_format(
            W2V_SOURCE, binary=True, unicode_errors="ignore"
        )
texts = [prep.norm_sentence(i) if i else [] for i in tqdm(row_data['title'])]


  0%|          | 0/119228 [00:00<?, ?it/s]

In [79]:
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(texts, min_count=20)
for idx in range(len(texts)):
    for token in bigram[texts[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            texts[idx].append(token)

In [91]:
bigram.freeze().save("/home/jovyan/work/phrases_model_last.pkl")

In [81]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(sentences=texts, vector_size=100, window=5, min_count=1, workers=4)
model.wv.save_word2vec_format("/home/jovyan/work/word2vec_moretech.bin", binary = True)

In [82]:
texts = [extract_keywords(i, model.wv, 10) for i in tqdm(texts)]

  0%|          | 0/119228 [00:00<?, ?it/s]

In [83]:
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(texts)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=3, no_above=0.5)


In [92]:
dictionary.save("/home/jovyan/work/dictionary_last.pkl")

In [85]:
corpus = [dictionary.doc2bow(doc) for doc in texts]

In [86]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 14534
Number of documents: 119228


In [87]:
# Train LDA model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 1000
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make an index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [88]:
import pyLDAvis.gensim_models as gensimvis

vis = gensimvis.prepare(model, corpus, dictionary)
pyLDAvis.display(vis)

/opt/conda/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/opt/conda/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.9/site-packages/joblib/backports.py:36: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(np.__version__) < '1.13':
/opt/conda/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version class

In [93]:
with open('/home/jovyan/work/lda_last.pkl', 'wb') as f:
     pickle.dump(model, f)

In [90]:
W2V_SOURCE = "/home/jovyan/work/word2vec_moretech.bin" # Путь к файлу custom_word2vec_uncod.bin
prep = Preparator()
w2v = KeyedVectors.load_word2vec_format(
            W2V_SOURCE, binary=True, unicode_errors="ignore"
        )
topics = [
    'нормативные проверки',
    'поставки',
    'изменение цен',
    'розничная торговля',
    'банки',
    'право',
    'рынок',
    'малый бизнес',
    'валюта',
    'судебные дела'
]
def get_topic(new):
    tokens = extract_keywords(prep.norm_sentence(new), w2v)
    bow = [i[1] for i in model.get_document_topics(dictionary.doc2bow(tokens))]
    return topics[np.array(bow).argmax()]

get_topic(  """
            Это какая-то новость о том, что у магазинов начались проблемы
            с продажей алкогольной продукции
            """)

'изменение цен'

In [ ]:
topics = [
    'нормативные проверки',
    'поставки',
    'изменение цен',
    'розничная торговля',
    'банки',
    'право',
    'рынок',
    'малый бизнес',
    'валюта',
    'судебные дела'
]

In [24]:
for idx, topic in model.print_topics(num_topics = 20):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.037*"получить" + 0.035*"акция" + 0.033*"проверка" + 0.031*"владелец" + 0.029*"оператор" + 0.029*"сделка" + 0.026*"производитель" + 0.025*"банкротство" + 0.024*"назвать" + 0.020*"форма"


Topic: 1 
Words: 0.071*"сбербанк" + 0.053*"товар" + 0.053*"интернет" + 0.048*"штраф" + 0.032*"продукт" + 0.027*"касса" + 0.023*"нарушение" + 0.023*"выплата" + 0.021*"исследование" + 0.020*"ндфл"


Topic: 2 
Words: 0.057*"цена" + 0.050*"фас" + 0.047*"закон" + 0.044*"рост" + 0.033*"доля" + 0.032*"планировать" + 0.023*"реклама" + 0.023*"квартал" + 0.023*"тысяча" + 0.023*"магнит"


Topic: 3 
Words: 0.088*"магазин" + 0.077*"продажа" + 0.072*"фнс" + 0.037*"минфин" + 0.035*"сми" + 0.028*"группа" + 0.025*"алкоголь" + 0.024*"прибыль" + 0.024*"торговля" + 0.020*"власть"


Topic: 4 
Words: 0.082*"цб" + 0.065*"рф" + 0.043*"предложить" + 0.028*"лицензия" + 0.026*"правительство" + 0.024*"правило" + 0.019*"счeт" + 0.018*"требование" + 0.018*"капитал" + 0.016*"смартфон"


Topic: 5 
Words: 0.056*"мос